In [ ]:
# Import class
import os
import sys
parent_dir = os.path.dirname(os.path.dirname(os.getcwd()))
class_dir = os.path.join(parent_dir, 'Scripts')
sys.path.insert(0, class_dir)
import datetime as dt
import scraper
import empirical
import pandas as pd

In [ ]:
best_markets = scraper.get_best_player_markets(dt.date(2019, 1, 29))

## Show markets and order by bookie edge

In [ ]:
bookie_pr_edge_list = []
for market in best_markets:
    bookie_pr_edge_list.append(market.calc_spread())

for edge, market in sorted(zip(bookie_pr_edge_list, best_markets), key=lambda t:t[0]):
    print('Bookie edge {:.2%}'.format(edge))
    print(market, end='\n\n')

## Calculate theos for best markets

In [ ]:
past_n_games = 40
edge_list = []
player_list = []
over_under_list = []
baseline_list = []
stat_list = []
odds_list = []
theo_list = []
bookie_edge_list = []
for p_mrkt in best_markets:
        
    try:
        # If unable to calculate theo (e.g. if player's name not correct), move onto next one
        # Calculate theo odds
        theo_odds_over, theo_odds_under = empirical.empirical_odds(p_mrkt.player, p_mrkt.stat, 
                                 p_mrkt.baseline, past_n_games)
    except:
        print('Unable to calculate theo for {} {}'.format(p_mrkt.player,
                                                      p_mrkt.stat))
        continue

    theo_pr_over = 1/theo_odds_over
    theo_pr_under = 1 - theo_pr_over

    # Calculate edge
    edge_over = theo_pr_over * p_mrkt.odds_over - 1
    edge_under = theo_pr_under * p_mrkt.odds_under - 1

    if edge_over > edge_under:
        best_edge = edge_over
        over_under = 'over'
        theo_list.append(theo_odds_over)
        odds_list.append(p_mrkt.odds_over)
    else:
        best_edge = edge_under
        over_under = 'under'
        theo_list.append(theo_odds_under)
        odds_list.append(p_mrkt.odds_under)

    bookie_edge = 1/float(p_mrkt.odds_over) + 1/float(p_mrkt.odds_under) - 1
    
    bookie_edge_list.append(bookie_edge)
    edge_list.append(best_edge)
    player_list.append(p_mrkt.player)
    over_under_list.append(over_under)
    baseline_list.append(p_mrkt.baseline)
    stat_list.append(p_mrkt.stat)
    
# Create df, sort by edge and print
df_game_mrkts_edge = pd.DataFrame({
    'Edge':edge_list,
    'Bookie_edge':bookie_edge_list,
    'Player':player_list,
    'over_under':over_under_list,
    'Baseline':baseline_list,
    'Stat':stat_list,
    'Odds':odds_list,
    'Theo':theo_list
}, columns = ['Edge', 'Bookie_edge', 'Player', 'over_under', 'Baseline', 'Stat', 'Odds', 'Theo'])

df_game_mrkts_edge = df_game_mrkts_edge.sort_values(by='Edge', ascending=False)
print(df_game_mrkts_edge)
print('')

## Record and save in pickle file

In [ ]:
import pickle

market_historic = [(dt.datetime.now(), best_markets)]
#pickle.dump(market_historic, open('historic_markets_20190129.pkl', 'wb'))